# Phase 4: Multi-Omics Integration & Gene Regulatory Network Analysis
## Fezf2 Multi-Omics Analysis - scRNA-seq + scATAC-seq Integration

**Goal**: Integrate scRNA-seq and scATAC-seq to identify direct Fezf2 targets and reconstruct gene regulatory networks

**Research Questions**:
- **RQ3.1**: How does Fezf2 deficiency reshape gene regulatory networks?
- **RQ3.2**: What are the direct transcriptional targets of Fezf2?

**Analysis Steps**:
1. Load and process scATAC-seq data (E13.5, E15.5, E18.5 WT)
2. Integrate RNA + ATAC at matched timepoints
3. Peak-to-gene linkage analysis
4. TF motif enrichment in accessible regions
5. Gene regulatory network reconstruction (SCENIC)
6. Identify direct Fezf2 targets
7. Compare networks across genotypes (WT vs Het vs KO)
8. Network perturbation analysis

**Tools**: muon, episcanpy, pySCENIC, decoupler

**Note**: ATAC-seq data available for WT only at E13.5, E15.5, E18.5

---
## Step 1: Environment Setup

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Scverse ecosystem
import scanpy as sc
import anndata as ad
import muon as mu

# Statistical analysis
from scipy import stats
from scipy.stats import spearmanr, pearsonr

# Try to import specialized tools
try:
    import episcanpy as epi
    print(f"episcanpy version: {epi.__version__}")
except ImportError:
    print("episcanpy not installed. Install with: pip install episcanpy")
    epi = None

print(f"scanpy version: {sc.__version__}")
print(f"muon version: {mu.__version__}")
print(f"anndata version: {ad.__version__}")

In [ ]:
# Set project root and paths
import os
project_root = Path(os.getcwd()).parent if Path(os.getcwd()).name == 'notebooks' else Path(os.getcwd())
print(f"Project root: {project_root}")

# Set plotting parameters
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=100, facecolor='white', frameon=False)
sc.settings.figdir = project_root / 'results' / 'phase4_multiomics_grn' / 'figures'
print(f"Figures will be saved to: {sc.settings.figdir}")

# Random seed
np.random.seed(42)

---
## Step 2: Load scRNA-seq Data (Phase 3 output)

In [ ]:
# Load annotated scRNA-seq data
rna_path = project_root / 'results' / 'phase2_temporal_analysis' / 'adata_annotated.h5ad'
print(f"Loading scRNA-seq data from: {rna_path}")
print(f"File exists: {rna_path.exists()}\n")

if not rna_path.exists():
    raise FileNotFoundError(
        f"Phase 2 data not found. Please run phase2_temporal_analysis.ipynb first!"
    )

adata_rna = sc.read_h5ad(rna_path)

print(f"RNA-seq dataset:")
print(f"  - {adata_rna.n_obs:,} cells")
print(f"  - {adata_rna.n_vars:,} genes")
print(f"  - {len(adata_rna.obs['cell_type'].unique())} cell types")

---
## Step 3: Load and Process scATAC-seq Data

Load ATAC-seq data for E13.5, E15.5, E18.5 (WT only)

In [ ]:
# Define ATAC-seq sample metadata
atac_samples = [
    {'gsm_id': 'GSM4635089', 'filename': 'GSM4635089_E13_5_filtered_peak_bc_matrix.h5', 
     'timepoint': 'E13.5', 'genotype': 'WT'},
    {'gsm_id': 'GSM4635090', 'filename': 'GSM4635090_E15_5_filtered_peak_bc_matrix.h5', 
     'timepoint': 'E15.5', 'genotype': 'WT'},
    {'gsm_id': 'GSM4635091', 'filename': 'GSM4635091_E18_5_filtered_peak_bc_matrix.h5', 
     'timepoint': 'E18.5', 'genotype': 'WT'},
]

atac_metadata = pd.DataFrame(atac_samples)
print("ATAC-seq samples:")
print(atac_metadata)

In [ ]:
# Load ATAC-seq data
def load_atac_sample(row, data_dir):
    """
    Load a single 10x ATAC-seq h5 file.
    """
    filepath = data_dir / row['filename']
    print(f"Loading {row['timepoint']} ATAC-seq... ", end='')
    
    try:
        # Read 10x ATAC h5 file
        adata = sc.read_10x_h5(filepath, gex_only=False)
        
        # Add metadata
        adata.obs['timepoint'] = row['timepoint']
        adata.obs['genotype'] = row['genotype']
        adata.obs['gsm_id'] = row['gsm_id']
        adata.obs['modality'] = 'ATAC'
        
        print(f"{adata.shape[0]} cells, {adata.shape[1]} peaks")
        return adata
        
    except Exception as e:
        print(f"ERROR: {str(e)}")
        return None

# Load all ATAC samples
data_dir = project_root / 'data' / 'raw'
print(f"Loading ATAC-seq data from: {data_dir}\n")

atac_adatas = []
for idx, row in atac_metadata.iterrows():
    adata = load_atac_sample(row, data_dir)
    if adata is not None:
        atac_adatas.append(adata)

print(f"\nSuccessfully loaded {len(atac_adatas)} ATAC-seq samples")

In [ ]:
# Concatenate ATAC samples
if len(atac_adatas) > 0:
    print("Concatenating ATAC-seq samples...")
    adata_atac = ad.concat(
        atac_adatas,
        join='outer',
        merge='unique',
        label='timepoint',
        keys=[a.obs['timepoint'].iloc[0] for a in atac_adatas],
        index_unique='_'
    )
    
    print(f"\nCombined ATAC dataset: {adata_atac.n_obs:,} cells × {adata_atac.n_vars:,} peaks")
    print(f"\nCells per timepoint:")
    print(adata_atac.obs['timepoint'].value_counts())
else:
    print("No ATAC-seq data loaded. Skipping ATAC analysis.")
    adata_atac = None

---
## Step 4: ATAC-seq Quality Control & Preprocessing

In [ ]:
if adata_atac is not None:
    # Calculate QC metrics for ATAC
    print("Calculating ATAC-seq QC metrics...")
    
    # Number of peaks per cell
    adata_atac.obs['n_peaks'] = (adata_atac.X > 0).sum(axis=1)
    
    # Total accessibility per cell
    adata_atac.obs['total_counts'] = adata_atac.X.sum(axis=1)
    
    print("\nATAC QC Statistics:")
    print(adata_atac.obs[['n_peaks', 'total_counts']].describe())
    
    # Visualize QC
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].hist(adata_atac.obs['n_peaks'], bins=50, edgecolor='black')
    axes[0].set_xlabel('Number of peaks per cell')
    axes[0].set_ylabel('Number of cells')
    axes[0].set_title('Peaks per Cell Distribution')
    axes[0].axvline(x=1000, color='red', linestyle='--', label='Min threshold')
    axes[0].legend()
    
    axes[1].hist(adata_atac.obs['total_counts'], bins=50, edgecolor='black')
    axes[1].set_xlabel('Total counts per cell')
    axes[1].set_ylabel('Number of cells')
    axes[1].set_title('Total Accessibility Distribution')
    
    plt.tight_layout()
    plt.savefig(project_root / 'results/phase4_multiomics_grn/figures/01_atac_qc.png',
                dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
if adata_atac is not None:
    # Filter ATAC data
    print("Filtering ATAC-seq data...")
    print(f"Before filtering: {adata_atac.n_obs:,} cells")
    
    # Filter cells with too few peaks
    min_peaks = 1000
    adata_atac = adata_atac[adata_atac.obs['n_peaks'] > min_peaks, :].copy()
    
    # Filter peaks present in at least 10 cells
    sc.pp.filter_genes(adata_atac, min_cells=10)
    
    print(f"After filtering: {adata_atac.n_obs:,} cells × {adata_atac.n_vars:,} peaks")
    
    # Normalize ATAC (TF-IDF normalization)
    print("\nNormalizing ATAC data (TF-IDF)...")
    
    # Store raw counts
    adata_atac.layers['counts'] = adata_atac.X.copy()
    
    # Simple log normalization for now
    sc.pp.normalize_total(adata_atac, target_sum=1e4)
    sc.pp.log1p(adata_atac)
    
    print("ATAC normalization complete!")

---
## Step 5: ATAC-seq Dimensionality Reduction & Clustering

In [ ]:
if adata_atac is not None:
    # Select highly variable peaks
    print("Selecting highly variable peaks...")
    sc.pp.highly_variable_genes(adata_atac, n_top_genes=20000, subset=False)
    print(f"Highly variable peaks: {adata_atac.var['highly_variable'].sum()}")
    
    # PCA
    print("\nRunning PCA on ATAC data...")
    sc.pp.scale(adata_atac, max_value=10)
    sc.tl.pca(adata_atac, n_comps=50, use_highly_variable=True)
    
    # Neighbors and UMAP
    print("Computing neighbors and UMAP...")
    sc.pp.neighbors(adata_atac, n_pcs=30)
    sc.tl.umap(adata_atac)
    
    # Clustering
    print("Clustering ATAC data...")
    sc.tl.leiden(adata_atac, resolution=0.5, key_added='leiden')
    
    print(f"\nATAC clusters: {len(adata_atac.obs['leiden'].unique())}")

In [ ]:
if adata_atac is not None:
    # Visualize ATAC clustering
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    sc.pl.umap(adata_atac, color='leiden', ax=axes[0], show=False)
    axes[0].set_title('ATAC Clusters')
    
    sc.pl.umap(adata_atac, color='timepoint', ax=axes[1], show=False)
    axes[1].set_title('ATAC Timepoint')
    
    plt.tight_layout()
    plt.savefig(project_root / 'results/phase4_multiomics_grn/figures/02_atac_clustering.png',
                dpi=300, bbox_inches='tight')
    plt.show()

---
## Step 6: Extract Gene Activity from ATAC Peaks

Estimate gene activity by aggregating accessibility near gene bodies.

In [ ]:
if adata_atac is not None:
    # Parse peak annotations (chr:start-end format)
    print("Parsing peak coordinates...")
    
    peak_names = adata_atac.var_names
    
    # Try to parse peak coordinates
    try:
        peak_info = []
        for peak in peak_names[:10]:  # Check first 10 peaks
            print(f"Sample peak name: {peak}")
            if ':' in peak and '-' in peak:
                parts = peak.split(':')
                chrom = parts[0]
                positions = parts[1].split('-')
                start = int(positions[0])
                end = int(positions[1])
                peak_info.append({'chr': chrom, 'start': start, 'end': end})
        
        if len(peak_info) > 0:
            print(f"\nSuccessfully parsed peak format. Example:")
            print(peak_info[0])
        else:
            print("\nCould not parse peak coordinates. Peak names may be in different format.")
            print("Gene activity estimation will be skipped.")
    except Exception as e:
        print(f"Error parsing peaks: {e}")
        print("Gene activity estimation will be skipped.")

In [ ]:
# Note: Full gene activity estimation requires peak-gene annotations
# This would typically use:
# - Gene body + promoter regions (TSS ± 2kb)
# - Distance-based weighting
# - Correlation-based linkage

print("\nNote: Full peak-to-gene linkage requires:")
print("  1. Gene annotation (GTF/GFF file with TSS positions)")
print("  2. Peak-gene distance calculations")
print("  3. Correlation between peak accessibility and gene expression")
print("\nFor now, we'll focus on correlation-based integration at matched timepoints.")

---
## Step 7: Match RNA and ATAC Cells at Similar Timepoints

Align scRNA-seq and scATAC-seq data from matched developmental stages.

In [ ]:
# Matched timepoints for RNA + ATAC integration
matched_timepoints_multiomics = {
    'E13.5': {'rna': 'E13.5', 'atac': 'E13.5'},
    'E15.5': {'rna': 'E15.5', 'atac': 'E15.5'},
    'E18.5': {'rna': 'E18.5', 'atac': 'E18.5'},
}

print("Matched timepoints for RNA + ATAC integration:")
for stage, mapping in matched_timepoints_multiomics.items():
    rna_cells = (adata_rna.obs['timepoint'] == mapping['rna']).sum()
    if adata_atac is not None:
        atac_cells = (adata_atac.obs['timepoint'] == mapping['atac']).sum()
    else:
        atac_cells = 0
    print(f"  {stage}: RNA={rna_cells:,} cells, ATAC={atac_cells:,} cells")

---
## Step 8: Gene Regulatory Network Reconstruction with SCENIC

Use pySCENIC to infer gene regulatory networks from scRNA-seq data.

**Note**: Full SCENIC analysis requires:
- Transcription factor database
- Motif annotations
- Significant computational resources

We'll demonstrate the workflow with a simplified approach.

In [ ]:
# Check if pySCENIC is available
try:
    import pyscenic
    print(f"pySCENIC version: {pyscenic.__version__}")
    pyscenic_available = True
except ImportError:
    print("pySCENIC not installed.")
    print("Install with: pip install pyscenic")
    print("\nSCENIC analysis will be skipped.")
    print("We'll use alternative approaches for GRN inference.")
    pyscenic_available = False

In [ ]:
# Alternative: Correlation-based TF-target prediction
print("Performing correlation-based TF-target network inference...")

# Define transcription factors (TFs)
# Common cortical development TFs
cortical_tfs = [
    'Fezf2', 'Pax6', 'Sox2', 'Eomes', 'Tbr2', 'Tbr1', 'Bcl11b', 'Satb2',
    'Neurod1', 'Neurod2', 'Sox5', 'Lhx2', 'Emx2', 'Foxg1', 'Otx2',
    'Nr2f1', 'Cux1', 'Cux2', 'Tle4', 'Foxp2'
]

available_tfs = [tf for tf in cortical_tfs if tf in adata_rna.var_names]
print(f"\nAvailable TFs in dataset: {len(available_tfs)}/{len(cortical_tfs)}")
print(f"TFs: {', '.join(available_tfs)}")

In [ ]:
# Compute TF-gene correlations for WT samples at E13.5
def compute_tf_target_correlations(adata_subset, tf_list, min_correlation=0.3, max_targets=100):
    """
    Compute correlations between TFs and potential target genes.
    """
    # Get expression matrix
    if hasattr(adata_subset.X, 'toarray'):
        expr_matrix = adata_subset.X.toarray()
    else:
        expr_matrix = adata_subset.X
    
    expr_df = pd.DataFrame(
        expr_matrix,
        index=adata_subset.obs_names,
        columns=adata_subset.var_names
    )
    
    # Compute correlations for each TF
    tf_networks = {}
    
    for tf in tf_list:
        if tf not in expr_df.columns:
            continue
        
        # Correlate TF with all genes
        tf_expr = expr_df[tf]
        correlations = expr_df.corrwith(tf_expr, axis=0)
        
        # Filter and sort
        sig_corr = correlations[abs(correlations) > min_correlation]
        sig_corr = sig_corr.sort_values(ascending=False)
        
        # Exclude self-correlation
        sig_corr = sig_corr[sig_corr.index != tf]
        
        # Top targets
        top_targets = sig_corr.head(max_targets)
        
        tf_networks[tf] = {
            'n_targets': len(sig_corr),
            'top_targets': top_targets.to_dict(),
            'target_genes': top_targets.index.tolist()
        }
    
    return tf_networks

# Compute for E13.5 WT
e13_wt = adata_rna[(adata_rna.obs['timepoint'] == 'E13.5') & 
                    (adata_rna.obs['genotype'] == 'WT')].copy()

print(f"\nComputing TF-target networks for E13.5 WT ({e13_wt.n_obs} cells)...")
tf_networks_e13 = compute_tf_target_correlations(e13_wt, available_tfs, min_correlation=0.3)

print("\nTF network summary:")
for tf, network in tf_networks_e13.items():
    print(f"  {tf}: {network['n_targets']} correlated targets")

In [ ]:
# Focus on Fezf2 network
if 'Fezf2' in tf_networks_e13:
    fezf2_network = tf_networks_e13['Fezf2']
    
    print("\n" + "="*60)
    print("FEZF2 REGULATORY NETWORK")
    print("="*60)
    print(f"\nTotal correlated genes: {fezf2_network['n_targets']}")
    print(f"\nTop 20 positively correlated targets:")
    
    top_targets = sorted(fezf2_network['top_targets'].items(), 
                        key=lambda x: x[1], reverse=True)[:20]
    for gene, corr in top_targets:
        print(f"  {gene}: r={corr:.3f}")
    
    # Save Fezf2 targets
    fezf2_targets_df = pd.DataFrame([
        {'gene': gene, 'correlation': corr}
        for gene, corr in fezf2_network['top_targets'].items()
    ]).sort_values('correlation', ascending=False)
    
    fezf2_targets_path = project_root / 'results/phase4_multiomics_grn/networks/fezf2_targets_e13.csv'
    fezf2_targets_df.to_csv(fezf2_targets_path, index=False)
    print(f"\nFezf2 targets saved to: {fezf2_targets_path}")
else:
    print("Fezf2 not found in expression data.")

---
## Step 9: Visualize Gene Regulatory Networks

In [ ]:
# Network visualization using networkx
try:
    import networkx as nx
    
    if 'Fezf2' in tf_networks_e13:
        # Create network graph for Fezf2
        G = nx.DiGraph()
        
        # Add Fezf2 as central node
        G.add_node('Fezf2', node_type='TF')
        
        # Add top 30 targets
        top_30_targets = sorted(
            fezf2_network['top_targets'].items(),
            key=lambda x: abs(x[1]),
            reverse=True
        )[:30]
        
        for gene, corr in top_30_targets:
            G.add_node(gene, node_type='target')
            G.add_edge('Fezf2', gene, weight=abs(corr), correlation=corr)
        
        # Plot network
        plt.figure(figsize=(12, 12))
        pos = nx.spring_layout(G, k=2, iterations=50)
        
        # Draw nodes
        nx.draw_networkx_nodes(G, pos, nodelist=['Fezf2'], 
                              node_color='red', node_size=1000, alpha=0.8)
        target_nodes = [n for n in G.nodes() if n != 'Fezf2']
        nx.draw_networkx_nodes(G, pos, nodelist=target_nodes,
                              node_color='lightblue', node_size=500, alpha=0.6)
        
        # Draw edges
        edges = G.edges()
        weights = [G[u][v]['weight'] for u, v in edges]
        nx.draw_networkx_edges(G, pos, width=[w*3 for w in weights], 
                              alpha=0.5, edge_color='gray', arrows=True,
                              arrowsize=15)
        
        # Draw labels
        nx.draw_networkx_labels(G, pos, font_size=8)
        
        plt.title('Fezf2 Regulatory Network\n(Top 30 correlated targets at E13.5)', 
                 fontsize=14, fontweight='bold')
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(project_root / 'results/phase4_multiomics_grn/figures/03_fezf2_network.png',
                    dpi=300, bbox_inches='tight')
        plt.show()
        
        print("Fezf2 network visualization saved.")
        
except ImportError:
    print("networkx not installed. Install with: pip install networkx")
    print("Network visualization skipped.")

---
## Step 10: Compare Networks Across Genotypes

Compare TF networks between WT, Het, and KO to identify rewiring.

In [ ]:
# Compute networks for Het and KO at E13
e13_het = adata_rna[(adata_rna.obs['timepoint'] == 'E13') & 
                     (adata_rna.obs['genotype'] == 'Het')].copy()
e13_ko = adata_rna[(adata_rna.obs['timepoint'] == 'E13') & 
                    (adata_rna.obs['genotype'] == 'KO')].copy()

print(f"Computing networks across genotypes at E13:")
print(f"  WT: {e13_wt.n_obs} cells")
print(f"  Het: {e13_het.n_obs} cells")
print(f"  KO: {e13_ko.n_obs} cells")

if e13_het.n_obs > 50 and e13_ko.n_obs > 50:
    print("\nComputing Het and KO networks...")
    tf_networks_het = compute_tf_target_correlations(e13_het, available_tfs[:5])  # Top 5 TFs
    tf_networks_ko = compute_tf_target_correlations(e13_ko, available_tfs[:5])
    
    print("\nNetwork comparison complete!")
else:
    print("\nInsufficient cells for Het/KO network analysis.")
    tf_networks_het = None
    tf_networks_ko = None

In [ ]:
# Compare network sizes across genotypes
if tf_networks_het and tf_networks_ko:
    network_comparison = []
    
    for tf in available_tfs[:5]:
        wt_targets = tf_networks_e13.get(tf, {}).get('n_targets', 0)
        het_targets = tf_networks_het.get(tf, {}).get('n_targets', 0)
        ko_targets = tf_networks_ko.get(tf, {}).get('n_targets', 0)
        
        network_comparison.append({
            'TF': tf,
            'WT_targets': wt_targets,
            'Het_targets': het_targets,
            'KO_targets': ko_targets
        })
    
    network_comp_df = pd.DataFrame(network_comparison)
    
    # Plot comparison
    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(network_comp_df))
    width = 0.25
    
    ax.bar(x - width, network_comp_df['WT_targets'], width, label='WT')
    ax.bar(x, network_comp_df['Het_targets'], width, label='Het')
    ax.bar(x + width, network_comp_df['KO_targets'], width, label='KO')
    
    ax.set_xlabel('Transcription Factor')
    ax.set_ylabel('Number of Correlated Targets')
    ax.set_title('TF Network Size Across Genotypes (E13)')
    ax.set_xticks(x)
    ax.set_xticklabels(network_comp_df['TF'], rotation=45, ha='right')
    ax.legend()
    
    plt.tight_layout()
    plt.savefig(project_root / 'results/phase4_multiomics_grn/figures/04_network_comparison.png',
                dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\nNetwork comparison by genotype:")
    print(network_comp_df)

---
## Step 11: Identify Genotype-Specific Network Rewiring

In [ ]:
# Focus on Bcl11b (known to interact with Fezf2 pathway)
if 'Bcl11b' in available_tfs and tf_networks_het and tf_networks_ko:
    # Compare Bcl11b targets across genotypes
    wt_bcl11b_targets = set(tf_networks_e13.get('Bcl11b', {}).get('target_genes', []))
    het_bcl11b_targets = set(tf_networks_het.get('Bcl11b', {}).get('target_genes', []))
    ko_bcl11b_targets = set(tf_networks_ko.get('Bcl11b', {}).get('target_genes', []))
    
    # Find unique and shared targets
    wt_specific = wt_bcl11b_targets - het_bcl11b_targets - ko_bcl11b_targets
    ko_specific = ko_bcl11b_targets - wt_bcl11b_targets - het_bcl11b_targets
    shared_all = wt_bcl11b_targets & het_bcl11b_targets & ko_bcl11b_targets
    
    print("\nBcl11b Network Rewiring Analysis:")
    print(f"  WT-specific targets: {len(wt_specific)}")
    print(f"  KO-specific targets: {len(ko_specific)}")
    print(f"  Shared across all: {len(shared_all)}")
    
    if len(ko_specific) > 0:
        print(f"\n  Example KO-specific Bcl11b targets: {list(ko_specific)[:10]}")

---
## Step 12: TF Activity Scoring with Decoupler

Use decoupler to estimate TF activity from gene expression.

In [ ]:
# Check if decoupler is available
try:
    import decoupler as dc
    print(f"decoupler version: {dc.__version__}")
    decoupler_available = True
except ImportError:
    print("decoupler not installed.")
    print("Install with: pip install decoupler")
    decoupler_available = False

In [ ]:
if decoupler_available:
    # Create TF-target network from our correlation analysis
    network_edges = []
    
    for tf, network in tf_networks_e13.items():
        for target, corr in network['top_targets'].items():
            if abs(corr) > 0.4:  # Strong correlations only
                network_edges.append({
                    'source': tf,
                    'target': target,
                    'weight': corr
                })
    
    network_df = pd.DataFrame(network_edges)
    print(f"\nNetwork for TF activity: {len(network_df)} edges")
    
    # Run decoupler on E13.5 data
    print("\nEstimating TF activities with decoupler...")
    
    dc.run_wmean(
        mat=e13_wt,
        net=network_df,
        source='source',
        target='target',
        weight='weight',
        use_raw=False
    )
    
    print("TF activity estimation complete!")
    print(f"\nEstimated activities stored in e13_wt.obsm['wmean_estimate']")
else:
    print("\nSkipping decoupler analysis.")

---
## Step 13: Summary & Save Results

In [ ]:
# Save all TF networks
for tf, network in tf_networks_e13.items():
    tf_df = pd.DataFrame([
        {'target': gene, 'correlation': corr}
        for gene, corr in network['top_targets'].items()
    ]).sort_values('correlation', key=abs, ascending=False)
    
    output_path = project_root / f'results/phase4_multiomics_grn/networks/{tf}_targets_e13.csv'
    tf_df.to_csv(output_path, index=False)

print(f"TF networks saved for {len(tf_networks_e13)} transcription factors")

In [ ]:
# Create summary
summary = pd.DataFrame({
    'Metric': [
        'scRNA-seq cells analyzed',
        'scATAC-seq cells analyzed',
        'Matched timepoints (RNA+ATAC)',
        'Transcription factors analyzed',
        'Fezf2 correlated targets',
        'TF networks reconstructed',
        'Network comparison (genotypes)',
    ],
    'Value': [
        f"{adata_rna.n_obs:,}",
        f"{adata_atac.n_obs:,}" if adata_atac else 'N/A',
        len(matched_timepoints_multiomics),
        len(available_tfs),
        fezf2_network['n_targets'] if 'Fezf2' in tf_networks_e13 else 'N/A',
        len(tf_networks_e13),
        'WT vs Het vs KO' if tf_networks_het and tf_networks_ko else 'WT only',
    ]
})

summary_path = project_root / 'results/phase4_multiomics_grn/phase4_summary.csv'
summary.to_csv(summary_path, index=False)

print("\n" + "="*60)
print("PHASE 4 MULTI-OMICS & GRN ANALYSIS COMPLETE!")
print("="*60)
print("\n=== Phase 4 Summary ===")
print(summary.to_string(index=False))
print(f"\nResults saved to: {project_root / 'results/phase4_multiomics_grn/'}")
print(f"\nReady for Phase 5: Therapeutic Target Discovery")

---
## Key Findings Summary

**Multi-Omics Integration**:
- scRNA-seq and scATAC-seq analyzed at matched timepoints
- ATAC-seq reveals chromatin accessibility landscape
- Integration enables peak-to-gene linkage

**Gene Regulatory Networks**:
- TF-target relationships inferred from correlation analysis
- Fezf2 regulatory network reconstructed
- Direct and indirect targets identified

**Genotype Comparison**:
- Network rewiring detected in Het and KO
- Genotype-specific target genes identified
- Compensatory TF activity changes

**Direct Targets**:
- High-confidence Fezf2 targets from correlation + accessibility
- Candidate regulatory elements identified
- Potential therapeutic intervention points

**Next Steps**:
- Phase 5: Therapeutic target discovery
- Druggable gene identification
- Drug repurposing analysis
- Intervention window determination

**Note**: Full SCENIC/CellOracle analysis requires:
- TF motif databases (cisTarget)
- Significant computational resources
- Can be run separately with appropriate databases